# Imports

In [17]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import DirectoryLoader
from langchain.indexes import SQLRecordManager, index
from pinecone import Pinecone, PodSpec
import os

from dotenv import load_dotenv
load_dotenv()

True

# Loading Data

In [4]:
loader = DirectoryLoader(
    path="./data", 
    glob="*.csv", 
    loader_cls=CSVLoader,
    show_progress=True)

docs = loader.load()

print(docs[20].page_content)

100%|██████████| 2/2 [00:00<00:00, 38.09it/s]

Title: Black Adam
Runtime: 125 minutes
Language: en
Overview: Nearly 5,000 years after he was bestowed with the almighty powers of the Egyptian gods—and imprisoned just as quickly—Black Adam is freed from his earthly tomb, ready to unleash his unique form of justice on the modern world.
Release Date: 2022-10-19
Genre: Action, Adventure, Science Fiction
Keyword: lightning, superhero, anti hero, based on comic, demon, superhero team, duringcreditsstinger, dc extended universe (dceu)
Recommendation: Black Panther: Wakanda Forever, Avatar: The Way of Water, The Woman King, M3GAN, Troll, Thor: Love and Thunder, Glass Onion: A Knives Out Mystery, Terrifier 2, Smile, Avatar, Top Gun: Maverick, The Guardians of the Galaxy Holiday Special, Guillermo del Toro's Pinocchio, Puss in Boots: The Last Wish, Bullet Train, Violent Night, Spider-Man: No Way Home, Ant-Man and the Wasp: Quantumania, Shazam! Fury of the Gods, Jeepers Creepers: Reborn, Fall
Cast: Raj Kala, Donny Carrington, James Cusati-Moye

# Splitting Data

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [6]:
print(splits[9].page_content)

Cast: Prakash Raj, Rajit Kapoor, Mehroos Mir, Mohd Talib, John Abraham, Nimish Desai, Ratna Pathak Shah, Kiran Kumar, Jacqueline Fernandez, Elham Ehsas, Babrak Akbari, Ranjit Punia, Jaimini Pathak, Habib Al Aidroos, Rakul Preet Singh, Serena Walia, Shahnawaz Bhatt
Crew: Rohit Chaturvedi, Lakshya Raj Anand, P. S. Vinod, Vishal Kapoor, Shreya Jain, Vijay Ganguly, Mohd Amin Khatib, Jubin Nautiyal, Nakul Chugh, Vaibhav Vishant, Sumit Batheja, Girish Nakod, Ken Metzker, Bishwadeep Dipak Chatterjee, Devanshu Asthana, Yogendra Mogre, Surbhi Yadav, Chavi Sirohi, Shashwat Sachdev, Aarif Sheikh, Mohd Talib, John Abraham, Kuldeep Yadav, Soumik Mukherjee, Kumaar, Minnakshi Das, Akshay Jayantilal Gada, Amritpal Singh, Dhaval Jayantilal Gada, Franz Spilhaus, Jayantilal Gada, Garima Mathur, Will Humphris, Vishal Mishra, Daniel Hansen, Ajay Kapoor
Stream: 
Buy: 
Rent:


# Creating Embeddings and Uploading to Pinecone

In [7]:
index_name = "film-bot-index"

# Create empty index
PINECONE_KEY, PINECONE_INDEX_NAME = os.getenv(
    'PINECONE_API_KEY'), os.getenv('PINECONE_INDEX_NAME')

pc = Pinecone(api_key=PINECONE_KEY)

# Uncomment if index is not created already
# pc.create_index(
#     name="film-bot-index",
#     dimension=1536,
#     metric="cosine",
#     spec=PodSpec(
#         environment="gcp-starter"
#     )
# )

# Target index and check status
pc_index = pc.Index(index_name)
print(pc_index.describe_index_stats())

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

vectorstore = PineconeVectorStore(
    pc_index, embeddings
)

# Create record manager
namespace = f"pinecone/{index_name}"
record_manager = SQLRecordManager(
    namespace, db_url="sqlite:///record_manager_cache.sql"
)

record_manager.create_schema()

{'dimension': 1536,
 'index_fullness': 0.00911,
 'namespaces': {'': {'vector_count': 911}},
 'total_vector_count': 911}


In [8]:
def _clear():
    """
    Hacky helper method to clear content.
    """
    index([], record_manager, vectorstore,
          cleanup="full", source_id_key="source")

# Uncomment this line if you want to clear the Pinecone vectorstore
_clear()

index(splits, record_manager, vectorstore,
      cleanup="full", source_id_key="source")

{'num_added': 911, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [9]:
query = "Movies based on novels or books."
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

Keyword: hero, superhero, space travel, based on comic, sequel, teenage girl, aftercreditsstinger, duringcreditsstinger, marvel cinematic universe (mcu), space adventure
Recommendation: The Hunger Games: The Ballad of Songbirds & Snakes, Wonka, Aquaman and the Lost Kingdom, The Killer, Napoleon, Five Nights at Freddy's, The Creator, Wish, Oppenheimer, Killers of the Flower Moon, Migration, Leave the World Behind, A Haunting in Venice, Captain Marvel, Barbie, Rebel Moon - Part One: A Child of Fire, The Equalizer 3, Saltburn, Lift, Elemental, Role Play


# Creating a Retriever

In [10]:
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 6})

In [11]:
retrieved_docs = retriever.invoke(
    "What are some films with about a person who becomes a tyrannical president of Panem?")

In [12]:
len(retrieved_docs)

6

In [13]:
print(retrieved_docs[0].page_content)

Title: The Hunger Games: The Ballad of Songbirds & Snakes
Runtime: 157 minutes
Language: en
Overview: 64 years before he becomes the tyrannical president of Panem, Coriolanus Snow sees a chance for a change in fortunes when he mentors Lucy Gray Baird, the female tribute from District 12.
Release Date: 2023-11-15
Genre: Drama, Science Fiction, Action
Keyword: daughter, based on novel or book, arena, fight, mentor, dystopia, riot, exploration, president, prequel, survival, murder, snow, creature, battle, fight to the death, young adult, origin story, based on young adult novel, death game, teenager
Recommendation: Aquaman and the Lost Kingdom, Napoleon, The Marvels, Wonka, The Hunger Games, Rebel Moon - Part One: A Child of Fire, The Family Plan, Oppenheimer, Thanksgiving, Killers of the Flower Moon, Five Nights at Freddy's, Saltburn, The Creator, There's Still Tomorrow, Silent Night, Barbie, Leave the World Behind, Wish, The Hunger Games: Mockingjay - Part 1, Family Switch, Leo


# Creating an LLM with Retriever

In [36]:
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms import HuggingFaceHub
from huggingface_hub.hf_api import HfFolder 

HfFolder.save_token(os.getenv('HUGGINGFACEHUB_API_TOKEN'))

REPO_ID = "HuggingFaceH4/zephyr-7b-beta"

llm = HuggingFaceHub(
    repo_id=REPO_ID,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

# chat_model = ChatHuggingFace(llm=llm)
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0.5)

prompt_template = PromptTemplate.from_template(
    """
    You are an expert on films. Your goal is to recommend films to users based on their
    question and the retrieved context. If you don't know have a recommendation, just say that you don't have one. 
    Format your film recommendations in a bullet point list. Always include the title of the film you are recommending,
    and always provide a justification for why you are recommending a particular film. You should recommend no more than
    five films. If the user asks for film recommendations based on runtime, provide the runtime length in minutes in your
    response. Only recommend films that are given to you in the context, do not recommend any other films.
    
    The documents that are provided will have the following format:

    ### FORMAT ###
    Title: [Title of film]
    Runtime: [Runtime of film] minutes
    Language: [Language of film in ISO 639-1 language code]
    Overview: [Overview of film]
    Release Date: [Release data of film in format YYYY-MM-DD]
    Genre: [Genre of film, a comma separated list]
    Keyword: [Keywords describing the film, a comma separated list]
    Recommendation: [Recommendations for other films based on the current film, a comma separated list]
    Cast: [Cast of film, a comma separated list]
    Crew: [Crew of film, a comma separated list]
    Stream: [Streaming provider to watch film, a comma separated list]
    Buy: [Provider to buy film, a comma separated list]
    Rent: [Provider to rent film, a comma separated list]
    ### FORMAT ###

    Question: {question} 

    Context: {context} 

    Answer:

    """
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | chat_model
    | StrOutputParser()
)

In [37]:
for chunk in rag_chain.stream("Recommend some films similar to Barbie."):
    print(chunk, end="", flush=True)

- Blue Beetle: This film is recommended because it shares the genre of adventure with Barbie and also features a protagonist with special abilities.
- Spider-Man: Across the Spider-Verse: This film is recommended because it shares the genre of adventure with Barbie and also features a superhero protagonist.
- Talk to Me: This film is recommended because it shares the genre of comedy with Barbie and also explores themes of communication and relationships.
- Gran Turismo: This film is recommended because it shares the genre of comedy with Barbie and also features a story centered around cars and racing.
- Indiana Jones and the Dial of Destiny: This film is recommended because it shares the genre of adventure with Barbie and also features a thrilling and action-packed story.

# Creating an Agent with Wiki + RAG

In [93]:
from typing import Optional
from langchain_core.callbacks import CallbackManagerForToolRun
from langchain.agents import AgentExecutor
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.tools import BaseTool

# Create tool from retriever
retriever_tool = create_retriever_tool(
    retriever,
    "movie_data",
    """
    Search for basic information regarding films. Information 
    includes: title, release date, original language, genre, 
    recommendations, keywords describing the film, the cast,
    crew, and where to stream, rent, or buy the film. If you
    need any of this information, use this tool!
    """
)

class CustomWikipediaQueryRun(BaseTool):
    """Tool that searches the Wikipedia API."""

    name: str = "wikipedia"
    description: str = (
        """
        A wrapper around Wikipedia. 
        Useful for when you need to answer general questions about 
        people, places, companies, facts, historical events, or other subjects. 
        Input should be a search query.
        """

    )
    api_wrapper: WikipediaAPIWrapper

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Wikipedia tool."""
        return self.api_wrapper.run(query)
    

# Create wikipedia tool
wikipedia_tool = CustomWikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools = [retriever_tool, wikipedia_tool]

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """
            You are an expert on films. Your goal is to recommend films to users based on their question and the retrieved context. 
            If you don't know have a recommendation, just say that you don't have one. Format your film recommendations in a bullet 
            point list. Always include the title of the film you are recommending, and always provide a justification for why you are 
            recommending a particular film. Your justification should be at least four sentences per film. You should recommend no more 
            than five films. If the user asks for film recommendations based on runtime, provide the runtime length in minutes in your response. 
            """
            ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(
            input_variables=['input'], template='{input}')),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_functions_agent(chat_model, tools, prompt_template)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [94]:
agent_executor.invoke(
    {"input": "Can you tell me runtime for the film Creed III?"})



> Entering new AgentExecutor chain...

Invoking: `movie_data` with `{'query': 'Creed III'}`


Title: Creed III
Runtime: 116 minutes
Language: en
Overview: After dominating the boxing world, Adonis Creed has thrived in his career and family life. When a childhood friend and former boxing prodigy, Damian Anderson, resurfaces after serving a long sentence in prison, he is eager to prove that he deserves his shot in the ring. The face-off between former friends is more than just a fight. To settle the score, Adonis must put his future on the line to battle Damian — a fighter with nothing to lose.
Release Date: 2023-03-01
Genre: Drama, Action
Keyword: husband wife relationship, philadelphia, pennsylvania, sports, deaf, sequel, orphan, former best friend, ex-con, childhood friends, juvenile detention center, boxing, prodigy

Recommendation: Avatar, Glass Onion: A Knives Out Mystery, Puss in Boots: The Last Wish, Black Panther: Wakanda Forever, Ant-Man and the Wasp: Quantumania, Black Adam,

{'input': 'Can you tell me runtime for the film Creed III?',
 'output': 'The runtime for the film "Creed III" is 116 minutes.\n\nHere are some film recommendations for you:\n\n1. Avatar: This film is a visually stunning epic that takes you on a journey to the alien world of Pandora. It combines breathtaking visuals with a compelling story about the clash between humans and the native inhabitants of the planet. The film\'s groundbreaking use of 3D technology immerses you in the world like never before.\n\n2. Black Panther: Wakanda Forever: This highly anticipated sequel to the original "Black Panther" film continues the story of the Black Panther and the nation of Wakanda. It explores themes of identity, power, and responsibility, while also delivering thrilling action sequences and a richly realized world. The film celebrates African culture and showcases a diverse cast of talented actors.\n\n3. John Wick: Chapter 4: If you\'re a fan of intense action films, then the "John Wick" series

# Using DataFrame Agent

In [87]:
from langchain_experimental.agents import create_pandas_dataframe_agent
import pandas as pd

# Specify the path to your CSV file
csv_2022 = './data/2022_movie_collection_data.csv'
csv_2023 = './data/2023_movie_collection_data.csv'

# Read the CSV file into a DataFrame
df_2022 = pd.read_csv(csv_2022)
df_2023 = pd.read_csv(csv_2023)

df_all = pd.concat([df_2022, df_2023])

display(df_all)

custom_prefix = """
You are an expert on films. Your goal is to recommend films to users based on their question and the retrieved context. 
If you don't know have a recommendation, just say that you don't have one. Format your film recommendations in a bullet 
point list. Always include the title of the film you are recommending, and always provide a justification for why you are 
recommending a particular film. Your justification should be at least four sentences per film. You should recommend no more 
than five films. If the user asks for film recommendations based on runtime, provide the runtime length in minutes in your response. """
agent = create_pandas_dataframe_agent(
    chat_model, 
    df_all,
    agent_type="openai-tools", 
    verbose=True, 
    prefix=custom_prefix,
    include_df_in_prompt=True, 
    number_of_head_rows=1)

agent.invoke(
    {
        "input": "Recommend films that have a similar relased date to Avatar: The Way of Water?"
    }
)

,Title,Runtime,Language,Overview,Release Date,Genre,Keyword,Recommendation,Cast,Crew,Stream,Buy,Rent
0,The Voyagers,108 minutes,it,Time jump to 1939 with teenager Max and his fr...,2022-11-21,"Fantasy, Adventure, TV Movie","time travel, zeitreise","Talk to Me, Jurassic World, Il mammone, Enemy ...","Federico Tocci, Gianmarco Saurino, Andrea Gaia...","Gabriele Scarfone, Marco Bassano, Gianni Vezzo...",NaN,NaN,NaN
1,The OctoGames,88 minutes,en,"Eight contestants compete in eight deadly, cla...",2022-10-07,"Thriller, Action, Horror","遊戲, 挑戰極限, 組團打怪","Patriots Time, Oppenheimer, The Boy and the He...","Luv Patel, Adam Pietripaoli, Fallon Gyurko, Al...","Galen Christy, Aaron Mirtes, Dylan A. Young, B...",NaN,"Amazon Video, Google Play Movies, YouTube, Vud...","Amazon Video, Google Play Movies, YouTube, Vud..."
2,Agent Game,90 minutes,en,"Harris, a CIA interrogator at an Agency black ...",2022-04-08,"Action, Thriller","investigation, murder, betrayal, conspiracy, g...","All the Old Knives, Avatar: The Way of Water, ...","Annie Ilonzeh, Dermot Mulroney, Matt Riedy, Bl...","Shanan Becker, Zachary King, William Bruce Joh...",Showtime Apple TV Channel,"Apple TV, Amazon Video, Google Play Movies, Yo...","Apple TV, Amazon Video, Google Play Movies, Yo..."
3,Attack,123 minutes,hi,"With the Parliament under siege, India’s first...",2022-04-01,"Action, Science Fiction, Thriller","parliament, super soldier, soldier, indian army","Napoleon, Lift, Sixty Minutes, The Marvels, Wi...","Prakash Raj, Rajit Kapoor, Mehroos Mir, Mohd T...","Rohit Chaturvedi, Lakshya Raj Anand, P. S. Vin...",NaN,NaN,NaN
4,Avatar: The Way of Water,192 minutes,en,Set more than a decade after the events of the...,2022-12-14,"Science Fiction, Adventure, Action","dying and death, loss of loved one, alien life...","Avatar, Glass Onion: A Knives Out Mystery, Pus...","Britain Dalton, Duane Evans Jr., Cliff Curtis,...","Mark Dobson, Aashrita Kamath, Evan Webber, Len...","Disney Plus, Max, Max Amazon Channel","Apple TV, Amazon Video, Google Play Movies, Yo...","Apple TV, Amazon Video, Google Play Movies, Yo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Ruby Gillman, Teenage Kraken",91 minutes,en,"Ruby Gillman, a sweet and awkward high school ...",2023-06-28,"Animation, Family, Fantasy, Comedy","high school, ocean, boat, transformation, fish...","The Flash, Barbie, Elemental, Transformers: Ri...","Jane Fonda, Tiffany Wu, Nicole Byer, Atticus S...","Bill Diaz, Tyler Phillips, Corey Kraft, Joseph...","Netflix, Netflix basic with Ads","Apple TV, Amazon Video, Google Play Movies, Yo...","Apple TV, Amazon Video, Google Play Movies, Yo..."
96,The Wandering Earth II,173 minutes,zh,Humans built huge engines on the surface of th...,2023-01-22,"Science Fiction, Action, Drama","space travel, space station, earth in peril, g...","The Wandering Earth, Fast X, Ant-Man and the W...","Li Peidong, Andy Friend, Liu Jiayun, Kawawa Ka...","Josh Reyes, Meng Jun, Xiangjin Yin, Albert Yeu...",Hoopla,"Apple TV, Amazon Video, Google Play Movies, Yo...","Apple TV, Amazon Video, Google Play Movies, Yo..."
97,Creed III,116 minutes,en,"After dominating the boxing world, Adonis Cree...",2023-03-01,"Drama, Action","husband wife relationship, philadelphia, penns...","A Man Called Otto, Avatar: The Way of Water, S...","Lamont Lankford, Todd Grisham, Jimmy Lennon Jr...","Karoline S. Menezes, Gregory Goodall, Sincere ...","Amazon Prime Video, fuboTV, MGM Plus Amazon Ch...","Apple TV, Amazon Video, Google Play Movies, Yo...","Apple TV, Amazon Video, Google Play Movies, Yo..."
98,Dungeons & Dragons: Honor Among Thieves,134 minutes,en,A charming thief and a band of unlikely advent...,2023-03-23,"Adventure, Fantasy, Comedy","platonic love, gang of thieves, dragon, role p...","The Super Mario Bros. Movie, Guardians of the ...","Justice Smith, Fionnlagh Allan, Georgia Lander...","Khalid Almeerani, Dawn Victoria Dudley, Armido...","Amazon Prime Video, fuboTV, MGM Plus Amazon Ch...","Apple TV, Amazon Vid



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Release Date'] == '2022-11-21']['Title'].values[0]"}`
responded: To recommend films with a similar release date to "Avatar: The Way of Water," I will retrieve the release date of "Avatar: The Way of Water" and search for films released around the same time. Please wait a moment while I fetch the information.

The Voyagers
Invoking: `python_repl_ast` with `{'query': "df[(df['Release Date'] > '2022-11-01') & (df['Release Date'] < '2022-12-01')]['Title'].values.tolist()"}`


['The Voyagers', 'Sri Asih', 'Black Panther: Wakanda Forever', 'Suzume', 'Glass Onion: A Knives Out Mystery', "Teddy's Christmas", 'That Time I Got Reincarnated as a Slime the Movie: Scarlet Bond', 'Disenchanted', "Teddy's Christmas", 'Lamborghini: The Man Behind the Legend', 'Enola Holmes 2']Here are some films that were released around the same time as "Avatar: The Way of Water":

- The Voyagers: This film follows teenager M

{'input': 'Recommend films that have a similar relased date to Avatar: The Way of Water?',
 'output': 'Here are some films that were released around the same time as "Avatar: The Way of Water":\n\n- The Voyagers: This film follows teenager Max and his friends as they navigate Mussolini\'s Rome in 1939 and search for Max\'s missing brother. It combines elements of fantasy, adventure, and time travel, making it a thrilling choice for fans of "Avatar: The Way of Water."\n\n- Sri Asih: This film is a horror-thriller set in Indonesia. It tells the story of a young woman who becomes possessed by a vengeful spirit. If you enjoy supernatural elements and suspenseful plots, "Sri Asih" might be a good choice.\n\n- Black Panther: Wakanda Forever: This highly anticipated sequel to "Black Panther" continues the story of T\'Challa and the kingdom of Wakanda. It explores themes of heroism, identity, and the power of community. If you enjoyed the world-building and action of "Avatar: The Way of Water,

# SQL

In [88]:
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///movie.db")
df_all.to_sql("movie", engine, index=False)

db = SQLDatabase(engine=engine)

agent_executor = create_sql_agent(chat_model, db=db, agent_type="openai-tools", verbose=True)

agent.invoke(
    {
        "input": "Recommend films that have a similar relased date to Avatar: The Way of Water?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Release Date'] == '2022-11-21']['Title'].values[0]"}`
responded: To recommend films that have a similar release date to "Avatar: The Way of Water," I will retrieve the release date of "Avatar: The Way of Water" and then search for films with a similar release date. Please wait a moment while I fetch the information.

The Voyagers
Invoking: `python_repl_ast` with `{'query': "df[df['Release Date'] == '2022-11-21']['Genre'].values[0]"}`


Fantasy, Adventure, TV Movie
Invoking: `python_repl_ast` with `{'query': "df[df['Release Date'] == '2022-11-21']['Runtime'].values[0]"}`


108 minutesBased on the release date of "Avatar: The Way of Water," I recommend the following films that have a similar release date:

1. The Voyagers:
   - Genre: Fantasy, Adventure, TV Movie
   - Runtime: 108 minutes
   - Justification: "The Voyagers" is a fantasy adventure film that shares a similar genre with "Avatar: The 

{'input': 'Recommend films that have a similar relased date to Avatar: The Way of Water?',
 'output': 'Based on the release date of "Avatar: The Way of Water," I recommend the following films that have a similar release date:\n\n1. The Voyagers:\n   - Genre: Fantasy, Adventure, TV Movie\n   - Runtime: 108 minutes\n   - Justification: "The Voyagers" is a fantasy adventure film that shares a similar genre with "Avatar: The Way of Water." It takes the audience on an exciting journey with teenagers as they navigate a different time period, just like the time jump in "Avatar: The Way of Water." The runtime of 108 minutes makes it a suitable choice for those who enjoy immersive fantasy adventures within a reasonable time frame.\n\nUnfortunately, I don\'t have information on other films with a similar release date to "Avatar: The Way of Water."'}